# Variogram Analysis with gslib-zero

This notebook covers:
1. Computing experimental variograms
2. Directional variograms for anisotropy
3. Building and evaluating variogram models
4. Exporting variogram parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gslib_zero import (
    gamv, GamvDirection,
    VariogramModel, VariogramType,
    plot_experimental, plot_model, plot_variogram,
    evaluate_variogram, export_variogram_par
)

## 1. Create Anisotropic Sample Data

We'll create data with stronger correlation in the E-W direction than N-S.

In [ ]:
np.random.seed(123)
n = 200

# Sample locations
df = pd.DataFrame({
    'x': np.random.uniform(0, 1000, n),
    'y': np.random.uniform(0, 1000, n),
    'z': np.zeros(n),
})

# Create anisotropic values (E-W trend stronger than N-S)
df['value'] = (
    0.01 * df.x +           # Strong E-W trend
    0.002 * df.y +          # Weaker N-S trend
    np.random.normal(0, 2, n)
)

print(f"Samples: {len(df)}")
print(f"Value range: {df.value.min():.2f} to {df.value.max():.2f}")

In [ ]:
# Visualize sample locations
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df.x, df.y, c=df.value, cmap='viridis', s=30)
plt.colorbar(scatter, label='Value')
plt.xlabel('X (Easting)')
plt.ylabel('Y (Northing)')
plt.title('Sample Locations')
plt.axis('equal')
plt.show()

## 2. Omnidirectional Variogram

In [ ]:
# Compute omnidirectional variogram
omni_result = gamv(
    df.x, df.y, df.z, df.value,
    nlag=20,
    lag_distance=40.0,
    binary=True
)[0]  # Single direction returns list of one

print(f"Lags: {len(omni_result.gamma)}")
print(f"Sample variance: {df.value.var():.3f}")

In [ ]:
# Plot omnidirectional variogram
ax = plot_experimental(omni_result, labels=['Omnidirectional'])
ax.axhline(df.value.var(), color='gray', linestyle='--', label='Sample Variance')
ax.legend()
ax.set_title('Omnidirectional Experimental Variogram')
plt.show()

## 3. Directional Variograms

Compute variograms in multiple directions to identify anisotropy.

In [ ]:
# Define directions: N-S (0°), E-W (90°), NE-SW (45°), NW-SE (135°)
directions = [
    GamvDirection(azimuth=0.0, azimuth_tolerance=22.5),    # N-S
    GamvDirection(azimuth=90.0, azimuth_tolerance=22.5),   # E-W
    GamvDirection(azimuth=45.0, azimuth_tolerance=22.5),   # NE-SW
    GamvDirection(azimuth=135.0, azimuth_tolerance=22.5),  # NW-SE
]

# Compute directional variograms
dir_results = gamv(
    df.x, df.y, df.z, df.value,
    nlag=15,
    lag_distance=50.0,
    directions=directions,
    binary=True
)

print(f"Computed {len(dir_results)} directional variograms")

In [ ]:
# Plot all directions
fig, ax = plt.subplots(figsize=(10, 6))

direction_labels = ['N-S (0°)', 'E-W (90°)', 'NE-SW (45°)', 'NW-SE (135°)']
colors = ['blue', 'red', 'green', 'orange']

for result, label, color in zip(dir_results, direction_labels, colors):
    ax.plot(result.lag_distances, result.gamma, 'o-', label=label, color=color)

ax.axhline(df.value.var(), color='gray', linestyle='--', alpha=0.5, label='Sill')
ax.set_xlabel('Lag Distance')
ax.set_ylabel('Gamma (Semivariance)')
ax.set_title('Directional Variograms - Anisotropy Analysis')
ax.legend()
ax.set_xlim(0, None)
ax.set_ylim(0, None)
plt.show()

## 4. Building Variogram Models

### Simple Spherical Model

In [ ]:
# Simple isotropic spherical model
simple_model = VariogramModel.spherical(
    sill=df.value.var() * 0.9,
    ranges=(200, 200, 10),
    nugget=df.value.var() * 0.1
)

print(f"Nugget: {simple_model.nugget:.3f}")
print(f"Sill: {simple_model.structures[0]['sill']:.3f}")
print(f"Total sill: {simple_model.total_sill:.3f}")

In [ ]:
# Plot model against omnidirectional variogram
ax = plot_variogram(
    experimental=omni_result,
    model=simple_model,
    title='Simple Spherical Model'
)
plt.show()

### Nested Model (Two Structures)

In [ ]:
# Nested model with short-range and long-range structures
nested_model = VariogramModel(nugget=0.5)
nested_model.add_structure(
    VariogramType.SPHERICAL,
    sill=2.0,
    ranges=(100, 100, 10),
)
nested_model.add_structure(
    VariogramType.EXPONENTIAL,
    sill=2.5,
    ranges=(400, 400, 20),
)

print(f"Structures: {len(nested_model.structures)}")
print(f"Total sill: {nested_model.total_sill:.3f}")

In [ ]:
ax = plot_variogram(
    experimental=omni_result,
    model=nested_model,
    title='Nested Model (Spherical + Exponential)'
)
plt.show()

### Anisotropic Model

In [ ]:
# Anisotropic model (longer range in E-W direction)
aniso_model = VariogramModel.spherical(
    sill=4.5,
    ranges=(400, 150, 10),  # Major (E-W), Minor (N-S), Vertical
    nugget=0.5,
    angles=(90.0, 0.0, 0.0)  # Azimuth 90° = E-W orientation
)

print(f"Major range (E-W): {aniso_model.structures[0]['ranges'][0]}")
print(f"Minor range (N-S): {aniso_model.structures[0]['ranges'][1]}")
print(f"Anisotropy ratio: {400/150:.2f}")

## 5. Evaluating Variogram Models

In [ ]:
# Evaluate model at specific distances
distances = np.linspace(0, 500, 100)
gamma_values = evaluate_variogram(simple_model, distances)

plt.figure(figsize=(10, 5))
plt.plot(distances, gamma_values, 'b-', linewidth=2)
plt.axhline(simple_model.total_sill, color='gray', linestyle='--', label='Sill')
plt.xlabel('Distance')
plt.ylabel('Gamma')
plt.title('Variogram Model Evaluation')
plt.legend()
plt.show()

## 6. Export Variogram Parameters

Export the variogram model in GSLIB parameter file format.

In [ ]:
# Export to GSLIB format
import tempfile
import os

with tempfile.NamedTemporaryFile(mode='w', suffix='.par', delete=False) as f:
    temp_path = f.name

export_variogram_par(nested_model, temp_path)

print("GSLIB Parameter File Format:")
print("=" * 40)
with open(temp_path) as f:
    print(f.read())

os.unlink(temp_path)

In [ ]:
# Export with inline comments
with tempfile.NamedTemporaryFile(mode='w', suffix='.par', delete=False) as f:
    temp_path = f.name

export_variogram_par(nested_model, temp_path, comment_style='inline')

print("With inline comments:")
print("=" * 40)
with open(temp_path) as f:
    print(f.read())

os.unlink(temp_path)

## Summary

This notebook demonstrated:
- Computing omnidirectional and directional variograms with `gamv()`
- Identifying anisotropy from directional variograms
- Building simple, nested, and anisotropic variogram models
- Plotting experimental variograms and models
- Evaluating models and exporting parameters